## Run this Notebook with Spark 2 1G kernel

#### Users MUST shutdown kernels!!!

In [1]:
#Ensure we are using the right kernel
print (sc.version)

2.2.0.cloudera1


In [2]:
import time
from itertools import islice

### Read Airlines Data

#### Read two CSV files from Linux space and count lines.  Note execution time

In [3]:
start_time = time.time()

readme_linux = sc.textFile("file:///home/kadochnikov/data/200*.csv")
print(readme_linux.count())

print("--- %s seconds ---" % (time.time() - start_time))

14462945
--- 21.53054118156433 seconds ---


#### You can execute your Linux scripts directly from the Notebook

In [4]:
!ls -l /home/kadochnikov/data/200*.csv

-rwxr-xr-x 1 kadochnikov kadochnikov 702878193 Apr  9  2015 /home/kadochnikov/data/2007.csv
-rwxr-xr-x 1 kadochnikov kadochnikov 689413344 Apr 10  2015 /home/kadochnikov/data/2008.csv


#### Read two CSV files from HDFS and count lines.  Note execution time

In [5]:
start_time = time.time()

readme_hdfs = sc.textFile("hdfs:///user/kadochnikov/Airlines/200*.csv")
print(readme_hdfs.count())

print("--- %s seconds ---" % (time.time() - start_time))

14462945
--- 5.052244663238525 seconds ---


#### You can execute your Linux > HDFS scripts directly from the Notebook

In [6]:
!hadoop fs -ls /user/kadochnikov/Airlines/200*.csv

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
-rw-r--r--   3 kadochnikov kadochnikov  702878193 2017-03-13 17:15 /user/kadochnikov/Airlines/2007.csv
-rw-r--r--   3 kadochnikov kadochnikov  689413344 2017-03-13 17:16 /user/kadochnikov/Airlines/2008.csv


#### Read single CSV file from HDFS space and count lines.

In [8]:
readme_raw = sc.textFile("hdfs:///user/kadochnikov/Airlines/2007.csv")
readme_raw.count()

7453216

#### Display first line

In [9]:
readme_raw.take(2)

['Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay',
 '2007,1,1,1,1232,1225,1341,1340,WN,2891,N351,69,75,54,1,7,SMF,ONT,389,4,11,0,,0,0,0,0,0,0']

#### Use the filter transformation to return a new RDD without header

In [10]:
readme_raw = sc.textFile("hdfs:///user/kadochnikov/Airlines/2007.csv")
readme = readme_raw.mapPartitionsWithIndex(lambda i, iter: islice(iter, 1, None) if i == 0 else iter) #filter out header
readme.count()

7453215

In [11]:
readme.take(10)

['2007,1,1,1,1232,1225,1341,1340,WN,2891,N351,69,75,54,1,7,SMF,ONT,389,4,11,0,,0,0,0,0,0,0',
 '2007,1,1,1,1918,1905,2043,2035,WN,462,N370,85,90,74,8,13,SMF,PDX,479,5,6,0,,0,0,0,0,0,0',
 '2007,1,1,1,2206,2130,2334,2300,WN,1229,N685,88,90,73,34,36,SMF,PDX,479,6,9,0,,0,3,0,0,0,31',
 '2007,1,1,1,1230,1200,1356,1330,WN,1355,N364,86,90,75,26,30,SMF,PDX,479,3,8,0,,0,23,0,0,0,3',
 '2007,1,1,1,831,830,957,1000,WN,2278,N480,86,90,74,-3,1,SMF,PDX,479,3,9,0,,0,0,0,0,0,0',
 '2007,1,1,1,1430,1420,1553,1550,WN,2386,N611SW,83,90,74,3,10,SMF,PDX,479,2,7,0,,0,0,0,0,0,0',
 '2007,1,1,1,1936,1840,2217,2130,WN,409,N482,101,110,89,47,56,SMF,PHX,647,5,7,0,,0,46,0,0,0,1',
 '2007,1,1,1,944,935,1223,1225,WN,1131,N749SW,99,110,86,-2,9,SMF,PHX,647,4,9,0,,0,0,0,0,0,0',
 '2007,1,1,1,1537,1450,1819,1735,WN,1212,N451,102,105,90,44,47,SMF,PHX,647,5,7,0,,0,20,0,0,0,24',
 '2007,1,1,1,1318,1315,1603,1610,WN,2456,N630WN,105,115,92,-7,3,SMF,PHX,647,5,8,0,,0,0,0,0,0,0']

#### Use the filter transformation to return a new RDD with a subset of the items in the file

In [12]:
lines_with_ord = readme.filter(lambda line: "ORD" in line)

In [13]:
lines_with_ord.take(10)

['2007,1,25,4,1052,1100,1359,1414,XE,1202,N12167,127,134,105,-15,-8,ORD,EWR,719,5,17,0,,0,0,0,0,0,0',
 '2007,1,20,6,845,845,1015,1020,XE,1201,N14153,150,155,123,-5,0,EWR,ORD,719,7,20,0,,0,0,0,0,0,0',
 '2007,1,28,7,1541,1500,1811,1750,XE,2836,N12163,150,170,128,21,41,ORD,IAH,925,7,15,0,,0,0,0,5,0,16',
 '2007,1,27,6,837,845,1013,1020,XE,1201,N12157,156,155,122,-7,-8,EWR,ORD,719,11,23,0,,0,0,0,0,0,0',
 '2007,1,29,1,2045,2000,2305,2211,XE,2701,N14508,80,71,52,54,45,ORD,CLE,316,9,19,0,,0,0,0,9,0,45',
 '2007,1,16,2,1044,1045,1229,1220,XE,1203,N14993,165,155,127,9,-1,EWR,ORD,719,8,30,0,,0,0,0,0,0,0',
 '2007,1,17,3,1851,1900,2204,2220,XE,1206,N19554,133,140,109,-16,-9,ORD,EWR,719,11,13,0,,0,0,0,0,0,0',
 '2007,1,12,5,2045,1745,2256,2003,XE,2023,N15973,71,78,45,173,180,ORD,CLE,316,6,20,0,,0,0,0,0,0,173',
 '2007,1,12,5,959,930,1247,1212,XE,2324,N11109,168,162,141,35,29,ORD,IAH,925,8,19,0,,0,0,0,6,0,29',
 '2007,1,26,5,2035,2000,2242,2211,XE,2701,N16510,67,71,46,31,35,ORD,CLE,316,6,15,0,,0,0,0,0,0,

#### Chain together transformations and actions. To find out how many lines contains "ORD", type in:

In [14]:
readme.filter(lambda line: "ORD" in line).count()

751500

#### Caching Airlines file to perform multiple filter operations on it

In [15]:
airlines_raw = sc.textFile("hdfs:///user/kadochnikov/Airlines/2007.csv")
airlines = airlines_raw.mapPartitionsWithIndex(lambda i, iter: islice(iter, 1, None) if i == 0 else iter)
airlines_orig_dest = airlines.map(lambda s: s.split(",")[16:18]) 
airlines_orig_dest.cache()

PythonRDD[21] at RDD at PythonRDD.scala:48

In [16]:
#First execution builds cache

start_time = time.time()

print (airlines_orig_dest.take(10))

print("--- %s seconds ---" % (time.time() - start_time))

[['SMF', 'ONT'], ['SMF', 'PDX'], ['SMF', 'PDX'], ['SMF', 'PDX'], ['SMF', 'PDX'], ['SMF', 'PDX'], ['SMF', 'PHX'], ['SMF', 'PHX'], ['SMF', 'PHX'], ['SMF', 'PHX']]
--- 10.786569833755493 seconds ---


In [17]:
#Second execution uses cached data

start_time = time.time()

print (airlines_orig_dest.take(10))

print("--- %s seconds ---" % (time.time() - start_time))

[['SMF', 'ONT'], ['SMF', 'PDX'], ['SMF', 'PDX'], ['SMF', 'PDX'], ['SMF', 'PDX'], ['SMF', 'PDX'], ['SMF', 'PHX'], ['SMF', 'PHX'], ['SMF', 'PHX'], ['SMF', 'PHX']]
--- 0.1302471160888672 seconds ---


In [18]:
airlines_orig_dest.filter(lambda line: "ORD" in line).count()

751500

In [19]:
airlines_orig_dest.filter(lambda line: "JFK" in line).count()

252704

In [20]:
airlines_orig_dest.filter(lambda line: "ORD" in line).take(10)

[['ORD', 'EWR'],
 ['EWR', 'ORD'],
 ['ORD', 'IAH'],
 ['EWR', 'ORD'],
 ['ORD', 'CLE'],
 ['EWR', 'ORD'],
 ['ORD', 'EWR'],
 ['ORD', 'CLE'],
 ['ORD', 'IAH'],
 ['ORD', 'CLE']]

In [21]:
airlines_orig_dest.filter(lambda line: "JFK" in line).take(10)

[['JFK', 'CLE'],
 ['CLE', 'JFK'],
 ['JFK', 'CLE'],
 ['JFK', 'CLE'],
 ['CLE', 'JFK'],
 ['JFK', 'CLE'],
 ['JFK', 'CLE'],
 ['JFK', 'CLE'],
 ['CLE', 'JFK'],
 ['CLE', 'JFK']]

#### Release the memory

In [49]:
airlines_orig_dest.unpersist()

PythonRDD[21] at RDD at PythonRDD.scala:48

#### Let's create two RDDs: one for origination and second for destination

In [27]:
airlines_raw = sc.textFile("hdfs:///user/kadochnikov/Airlines/2007.csv")
#airlines = airlines_raw.mapPartitionsWithIndex(lambda i, iter: islice(iter, 1, None) if i == 0 else iter)
airlines_orig = airlines_raw.map(lambda s: s.split(",")[16]) 
airlines_dest = airlines_raw.map(lambda s: s.split(",")[17]) 

In [28]:
airlines_orig.first()

'Origin'

In [29]:
airlines_dest.first()

'Dest'

#### In this map invocation, we use a function which replaces each original value in the input RDD with a 2-tuple containing the value (origin) in the first position and the integer value 1 in the second position:

In [30]:
a1 = airlines_orig.map(lambda x: (x, 1))

In [31]:
a1.take(10)

[('Origin', 1),
 ('SMF', 1),
 ('SMF', 1),
 ('SMF', 1),
 ('SMF', 1),
 ('SMF', 1),
 ('SMF', 1),
 ('SMF', 1),
 ('SMF', 1),
 ('SMF', 1)]

#### Here the key will be the word and lambda function will sum up the word counts for each word. The output RDD will consist of a single tuple for each unique word in the data, where the word is stored at the first position in the tuple and the word count is stored at the second position

In [32]:
a2 = a1.reduceByKey(lambda x,y:x+y)
a2.take(10)

[('ONT', 41643),
 ('PDX', 58603),
 ('SWF', 5592),
 ('AUS', 51560),
 ('BQN', 1370),
 ('FLL', 69414),
 ('LGB', 14210),
 ('DCA', 89666),
 ('IAH', 200420),
 ('DTW', 177478)]

#### Map a lambda function to the data which will swap over the first and second values in each tuple, now the word count appears in the first position and the word in the second position

In [33]:
a3 = a2.map(lambda x:(x[1],x[0]))
a3.take(10)

[(177478, 'DTW'),
 (58603, 'PDX'),
 (200420, 'IAH'),
 (51560, 'AUS'),
 (89666, 'DCA'),
 (69414, 'FLL'),
 (41643, 'ONT'),
 (5701, 'ABE'),
 (23535, 'BHM'),
 (6171, 'MOB')]

#### Sort the input RDD by the key value (the value at the first position in each tuple)

In [34]:
a4 = a3.sortByKey(ascending=False)
a4.take(10)

[(413851, 'ATL'),
 (375784, 'ORD'),
 (297345, 'DFW'),
 (240928, 'DEN'),
 (237597, 'LAX'),
 (211072, 'PHX'),
 (200420, 'IAH'),
 (183668, 'LAS'),
 (177478, 'DTW'),
 (155846, 'MSP')]

#### Calculate frequencies by airport for origination and destination

In [35]:
a_orig = airlines_orig.map(lambda x: (x, 1)).reduceByKey(lambda x,y:x+y)
a_dest = airlines_dest.map(lambda x: (x, 1)).reduceByKey(lambda x,y:x+y)

#### The join function combines the two datasets (K,V) and (K,W) together and get (K, (V,W)).

In [36]:
a_orig_dest = a_orig.join(a_dest)
a_orig_dest.cache()

PythonRDD[62] at RDD at PythonRDD.scala:48

In [37]:
a_orig_dest.take(10)

[('AUS', (51560, 51579)),
 ('BHM', (23535, 23540)),
 ('DTW', (177478, 177471)),
 ('FLL', (69414, 69444)),
 ('ONT', (41643, 41644)),
 ('PDX', (58603, 58634)),
 ('AEX', (2996, 2988)),
 ('IAH', (200420, 200428)),
 ('BFL', (5195, 5201)),
 ('LRD', (2543, 2542))]

#### Combine the values together to get the total count

In [38]:
a_orig_dest_sum = a_orig_dest.map(lambda k: (k[0], (k[1][0]+k[1][1])))

In [39]:
a_orig_dest_sum.take(10)

[('AUS', 103139),
 ('BHM', 47075),
 ('DTW', 354949),
 ('FLL', 138858),
 ('ONT', 83287),
 ('PDX', 117237),
 ('AEX', 5984),
 ('IAH', 400848),
 ('BFL', 10396),
 ('LRD', 5085)]

In [50]:
#Free-up memory
a_orig_dest.unpersist()
airlines_orig.cache()

PythonRDD[112] at RDD at PythonRDD.scala:48

## reduceByKey in detail

#### Looking at the results of map function

In [51]:
a1 = airlines_orig.map(lambda x: (x, 1)).cache()
print (a1.take(10))

[('Origin', 1), ('SMF', 1), ('SMF', 1), ('SMF', 1), ('SMF', 1), ('SMF', 1), ('SMF', 1), ('SMF', 1), ('SMF', 1), ('SMF', 1)]


#### Applying a "textbook" variant of reduceByKey

In [52]:
b1 = a1.reduceByKey(lambda x,y:x+y)
print (b1.take(10))

[('DTW', 177478), ('PDX', 58603), ('IAH', 200420), ('AUS', 51560), ('DCA', 89666), ('FLL', 69414), ('ONT', 41643), ('ABE', 5701), ('BHM', 23535), ('MOB', 6171)]


#### Replacing (x+y) with a traditional (x+1) counter

In [53]:
b2 = a1.reduceByKey(lambda x,y:x+1)
print (b2.take(10))

[('AUS', 9028), ('BHM', 4545), ('DTW', 31171), ('FLL', 13269), ('ONT', 7822), ('PDX', 11147), ('AEX', 841), ('IAH', 44626), ('BFL', 1377), ('LRD', 573)]


#### What if we use (y+1) as counter instead

In [54]:
b3 = a1.reduceByKey(lambda x,y:y+1)
print (b3.take(10))

[('IAH', 3), ('LRD', 3), ('LCH', 3), ('DCA', 3), ('AUS', 3), ('MOB', 3), ('DTW', 3), ('BHM', 3), ('SHV', 3), ('ONT', 3)]


#### Now back to the textbook example of (x+y)

In [55]:
b4 = a1.reduceByKey(lambda x,y:x+y)
print (b4.take(10))

[('DCA', 89666), ('DTW', 177478), ('FLL', 69414), ('IAH', 200420), ('ONT', 41643), ('PDX', 58603), ('ABE', 5701), ('AUS', 51560), ('BHM', 23535), ('MOB', 6171)]


#### Compare results for one airport (Gerald R. Ford International Airport)

In [56]:
#Reduced by x+y
c1 = b1.filter(lambda line: "GRR" in line)
print (c1.take(10))

#Reduced by x+1
c2 = b2.filter(lambda line: "GRR" in line)
print (c2.take(10))

#Reduced by y+1
c3 = b3.filter(lambda line: "GRR" in line)
print (c3.take(10))

[('GRR', 16412)]
[('GRR', 697)]
[('GRR', 3)]


#### What is the highest possible number of flights for any airport using (y+1) counter

In [57]:
#Reduced by y+1
d1 = b3.map(lambda x:(x[1],x[0]))
d2 = d1.sortByKey(ascending=False)

print (d2.take(10))

[(3, 'XNA'), (3, 'MKE'), (3, 'GSP'), (3, 'RIC'), (3, 'PVD'), (3, 'IAD'), (3, 'LIT'), (3, 'SAV'), (3, 'CMH'), (3, 'PIT')]


#### Now let's look at the highest results for proper (x+y) counter.  Do they look close to the US busiest airports?

In [58]:
#Reduced by x+1
d3 = b1.map(lambda x:(x[1],x[0]))
d4 = d3.sortByKey(ascending=False)

print (d4.take(10))

[(413851, 'ATL'), (375784, 'ORD'), (297345, 'DFW'), (240928, 'DEN'), (237597, 'LAX'), (211072, 'PHX'), (200420, 'IAH'), (183668, 'LAS'), (177478, 'DTW'), (155846, 'MSP')]


###  reduceByKey explanation

reduceByKey is an associative and commutative reduce function
reduceByKey operates by merging the values for each key using an associative and commutative reduce function.  
It applies the function first within each partition on a per-key basis and then across the partitions, allowing it to scale efficiently to large datasets
(x+1) only provides us with results within one of the partitions
(y+1) only provides us with results across partitions (max value = # of partitions)
(x+y) combines results within each partition with results across all partitions, by computing local sums for each key in each partition and combining those local sums into larger sums after shuffling

### Save results
Spark (just like Pig) expects empty directory and will err-out if the directory exists

In [67]:
!hadoop fs -ls "/user/kadochnikov/temp"

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
Found 3 items
-rw-r--r--   3 kadochnikov kadochnikov          0 2018-04-29 17:50 /user/kadochnikov/temp/_SUCCESS
-rw-r--r--   3 kadochnikov kadochnikov      19475 2018-04-29 17:50 /user/kadochnikov/temp/part-00000
-rw-r--r--   3 kadochnikov kadochnikov      19394 2018-04-29 17:50 /user/kadochnikov/temp/part-00001


In [68]:
!hadoop fs -rm -r "/user/kadochnikov/temp"

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
2018-04-29 20:18:03,256 INFO  [main] fs.TrashPolicyDefault (TrashPolicyDefault.java:moveToTrash(163)) - Moved: 'hdfs://nameservice1/user/kadochnikov/temp' to trash at: hdfs://nameservice1/user/kadochnikov/.Trash/Current/user/kadochnikov/temp


In [69]:
d4.saveAsTextFile("/user/kadochnikov/temp/")
#Won't work due to bug (or feature) in our Spark configuration
#wordCounts.saveAsTextFile("/home/kadochnikov/temp/") 

In [79]:
#!mkdir '/home/kadochnikov/temp/'

In [80]:
#Getmerge will overwrite the results
!hadoop fs -getmerge '/user/kadochnikov/temp' '/home/kadochnikov/temp/air_frequency.txt'

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0


In [82]:
!ls -l '/home/kadochnikov/temp'

total 32
-rw-r--r-- 1 kadochnikov kadochnikov 4332 Apr 29 20:21 air_frequency.txt


In [83]:
#Cleaning-up to eliminate wasted disk space
!hadoop fs -rm -r "/user/kadochnikov/test"

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
rm: `/user/kadochnikov/test': No such file or directory


In [84]:
#Releasing memory
airlines_orig.unpersist()

PythonRDD[112] at RDD at PythonRDD.scala:48